<img style="float: left;" src="earth-lab-logo-rgb.png" width="150" height="150" />

# Earth Analytics Python Course: Spring 2020

Before submitting this assignment, be sure to restart the kernel and run all cells. To do this, pull down the Kernel drop down at the top of this notebook. Then select **restart and run all**.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below.

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Homework 2 - Lidar & Raster Data
## About the 2013 Colorado Floods

The data that you will use for this assignment is the same data that you used in the last assignment. Use: `et.data.get_data("colorado-flood")` to download the data if you do not already have it. 

Using the data from the directory as follows:

In the `colorado-flood/spatial/boulder-leehill-rd/` you will see two directories - pre and post flood. each of these directories contains a lidar sub directory that has a DTM and a DSM.tif file. You will use these files to create your lidar CHM. 


For all of the plots below, be sure to do the following:
* State the source of each dataset that you use to create the plot either as a plot label or in a caption below the plot
* Make sure each plot has a clear TITLE and, where appropriate, label the x and y axes. Be sure to include UNITS in your labels!
* Analyze / interpret what each plot shows in a markdown cell in your notebook. State what the source of the data are and what the plot shows as a driver or impact of the 2013 flood events.
* Clearly document that steps that you took to process the data by commenting your code
* Use clean coding practices - this includes comments, variable names that are informative, clean code spacing, following PEP 8 Python code style guide.
* Make sure all of the libraries that you use in the report are listed in a code cell at the TOP of your Notebook.
* Spell check your Notebook before submitting it.
* Include a markdown cell for each histogram that briefly describes how you selected the classification values used in the raster plots referencing the histograms of the data that you create.



### Write Up
In your notebook, create markdown cells below each raster plot that describes what your classification map shows about changes in the landscape before and after the flood.

Be sure to spell check and proof your text before submitting!


### DATA
You can download the data from earthpy using the following syntax:

`et.data.get_data("colorado-flood")`

In [ ]:
# Imports for autograding - do not edit!
import matplotcheck.base as mpc
import matplotcheck.autograde as ag
import matplotcheck.notebook as nb
import matplotcheck.raster as ra

## PLOT 1: pre/post CHM difference raster histogram (15 points)

Create a cropped raster map that shows positive and negative change in the canopy height model before and after the flood. To do this:

* Subtract the pre-flood CHM from the post-flood CHM (post_flood_CHM - pre_flood_CHM).
* Crop the data using the data/colorado-flood/spatial/boulder-leehill-rd/clip-extent.shp`crop_extent shapefile.

NOTE: I am asking you to subtract the pre-conditions from the post conditions to ensure that your negative values  represent loss of terrain. This will be easier for you to explain in your writeup! 

IMPORTANT: 
* Name your difference CHM object: `diff_chm_crop`.
* Set your bin values to 30 for your plot.


In [ ]:
# Import libraries here - be sure to only list packaged used in this assignment 
# Use pep 8 guidelines for the package order. 10 points total for imports + pep 8 throughout

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# In this cell, create the difference CHM & DTM objects - it should be a numpy array.
# IMPORTANT: Name this object diff_chm_crop

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Let's make sure you created an object with the correct name and of the correct type above! (5 Points)
try:
    isinstance(diff_chm_crop, np.ndarray)
    print("Great! Your diff chm object is the correct type - np.ndarray")
except AssertionError as message:
    print("AssertionError:",
          "Oops, your clipped image should be a numpy array.")

## Plot 1: Histogram of the cropped data

In [ ]:
# IMPORTANT: Do not use plt.show in this cell. Add your code to produce plot 1 here

# YOUR CODE HERE
raise NotImplementedError()

### DO NOT REMOVE LINE BELOW ###
plot01_chm_diff_hist = nb.convert_axes(plt, which_axes="current")

## What does the histogram show you? (8 points)
Answer the following questions:

1. What do each bar in the histogram plot above represent?
2. What does this histogram tell you about differences in the CHM and in the landscape before vs after the flood?
3. What is the source of the data that you used to create this plot?
 

YOUR ANSWER HERE

## Plot 2: Classified pre/post CHM difference raster (14 points)

In the cell below, create a plot of the classified the lidar Canopy Height Model raster that you used to create the histogram above. 

Use the following bins to classify your data:

* `< -1`
* `between -1 and 1` 
* `> 1`

Compare your output plotted raster with your histogram.

Add a legend to your plot with the following categories:

* Loss
* No Change
* Gain

These categories should map to the bins that you used to classified the data

In [ ]:
# IMPORTANT: Name your classified raster object diff_chm_class
# IMPORTANT: Do not use plt.show in this cell.

# YOUR CODE HERE
raise NotImplementedError()

### DO NOT REMOVE LINE BELOW ###
plot02_chm_diff_map = nb.convert_axes(plt, which_axes="current")

# Manually Graded elements
* Legend has correct labels and colors (3 points)
* X and y axis labels and ticks are not displayed (3 points)

YOUR ANSWER HERE

## Plot 3: pre/post DTM Difference Raster Histogram (14 points)

Create a cropped raster map that shows positive and negative change in the digital terrain model before and after the flood. To do this:

* Subtract the pre-flood DTM from the post-flood DTM (post_flood_DTM - pre_flood_DTM).
* Crop the data using the data/colorado-flood/spatial/boulder-leehill-rd/clip-extent.shp` crop_extent shapefile.

Plot a histogram of the cropped pre/post DTM data that you created above. Set your bin number to 30 and be sure to label the x and y axis with appropriate units!

In [ ]:
# IMPORTANT: Do not use plt.show in this cell. Add your code for plot 4 here

# YOUR CODE HERE
raise NotImplementedError()

### DO NOT REMOVE LINE BELOW ###
plot_3_answer = nb.convert_axes(plt, which_axes="current")

### Plot 4: pre/post DTM difference raster (15 points)

Reclassify your difference DTM using the following range values:

* `< -1`
* `between -1 and 1` 
* `> 1`

Plot your difference DTM raster using matplotlib.
Create a legend for your plot that has 3 categories: 

* Loss
* No Change
* Gain

Each category should represent one of the "bins" that you used when you classified your data. 

In [ ]:
# IMPORTANT: Name your classified raster object diff_dtm_crop_class
# IMPORTANT: Do not use plt.show in this cell.

# YOUR CODE HERE
raise NotImplementedError()

### DO NOT REMOVE LINE BELOW ###
plot_4_answer = nb.convert_axes(plt, which_axes='current')

# Manually Graded plot elements (SKIP THIS CELL)
* Legend has correct labels and colors (2 points)
* X and y axis labels and ticks are not displayed (1 points)

##  Question (7 points)

* Look at the CHM plot and compare it to the DTM difference plot that you created above
* Consider how the data were collected, what they show and how they were processed? What factors could impact and add uncertainty to the CHM difference map?

Add your answer below using markdown 

YOUR ANSWER HERE